In [41]:
import numpy as np
import pandas as pd
import math
import os
import sklearn.preprocessing
import markov_clustering as mc
from scipy.sparse import isspmatrix
np.set_printoptions(precision=1,suppress=True)

In [42]:
mail = pd.read_csv('mails.csv')

source_set = set(mail['source'])
target_set = set(mail['target'])
num_people = len(source_set.union(target_set))

G = np.zeros((num_people,num_people))
for i, m in mail.iterrows():
    G[m.source,m.target] = m.weight
    G[m.target,m.source] = m.weight

In [43]:
class MCL:
    def __init__(self, G, e, r, loop_value=1):
        self.T = G
        np.fill_diagonal(self.T, loop_value)
        self.e = e
        self.r = r
        
    def normalize(self):
        self.T = sklearn.preprocessing.normalize(self.T, norm='l1', axis=0)
        
    def inflate(self):
        self.T = np.power(self.T, self.r)
    
    def expand(self):
        self.T = np.linalg.matrix_power(self.T, self.e)
    
    def converged(self):
        return np.allclose(self.T, self.G)
        
    def run(self):
        print("="*30)
        print("Inflation Factor:", self.r)
        print("Expansion Factor:", self.e)
        print("Input Graph:")
        print(self.T)
        print("="*30)
        
        self.normalize()
        self.G = self.T.copy()
        while True:
            self.expand()
            self.inflate()
            self.normalize()
            if self.converged():
                break
            self.G = self.T.copy()
    
    def get_clusters(self):
        clusters = set()
        for row in set(self.T.nonzero()[0]):
            clusters.add(tuple(self.T[row].nonzero()[0]))
        return clusters
        
    def get_matrix(self):
        return self.T
    
    def to_json(self):
        export = []
        for row in range(self.T.shape[0]):
            for column in range(self.T.shape[1]):
                if self.T[row, column] > 1e-4:
                    export.append([row, column, self.T[row, column]])
        export = pd.DataFrame(export,columns=['source', 'target', 'weight'])
        return export.to_json(orient='records')
        

In [44]:
M = MCL(G, 2, 3.1) # optimal modularity
M.run()
print(M.get_clusters())
print(M.to_json())

Inflation Factor: 3.1
Expansion Factor: 2
Input Graph:
[[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 2. 2. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 1. 0. 0. 4. 5. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 1. 0. 0. 0. 0. 6. 4. 5. 7. 4. 5. 0. 0. 0.]
 [0. 0. 3. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 3. 5.]
 [0. 0. 0. 4. 0. 0. 1. 3. 4. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 5. 0. 0. 3. 1. 4. 0. 0. 0. 0. 0. 0. 0. 2. 1.]
 [0. 0. 0. 5. 0. 0. 4. 4. 1. 0. 0. 0. 0. 0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 6. 0. 0. 0. 0. 1. 2. 4. 7. 2. 4. 0. 2. 0.]
 [0. 0. 0. 0. 4. 0. 0. 0. 0. 2. 1. 5. 5. 4. 3. 0. 2. 0.]
 [0. 0. 0. 0. 5. 0. 0. 0. 0. 4. 5. 1. 3. 4. 0. 0. 2. 0.]
 [0. 0. 0. 0. 7. 0. 0. 0. 0. 7. 5. 3. 1. 0. 3. 0. 0. 0.]
 [0. 0. 0. 0. 4. 0. 0. 0. 0. 2. 4. 4. 0. 1. 0. 0. 2. 0.]
 [0. 0. 0. 0. 5. 0. 0. 0. 0. 4. 3. 0. 3. 0. 1. 0. 2. 0.]
 [0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 3. 4.]
 [0. 0. 0. 0. 0. 3. 2. 2. 3. 2. 2